###This Project aims to detect a certain logo in an ID and remove it. We explored various approaches like YOLO v8, semantic segmentation and finally SIFT. We introduce our solution using SIFT here, followed by inpainting to remove background.

###Credits go to my colleague in the track Salma Mahran for pointing me in the SIFT direction.

Since we need to output a function, we organized our way like this. With Process_image as our main funcion, it takes 3 arguements which are the paths of logo, input and output.

While we could have made a function with 2 arguments only as instructed, we trust that adding the location of the logo as a variable will help make the function robust and agile to many label - image detections in the future. Otherwise, we suggest giving it a default value at function creation to behave as a 2 argument function.

Moreover, the function handles cases where any of the input paths is invalid or does not exist.

The function also takes a single image or a folder of images as an input to process bulk images in one execution and save time and effort.

We made sure that the function only saves the images where the logo has been detected, output an accuracy score and print the names of images where the label was not detected.


First we install dependencies

In [69]:
# #Library installation commands

# !pip install opencv-python
# !pip install numpy

In [70]:
import cv2
import numpy as np
import os

This is our input paths validation function

In [71]:
VALID_IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp') #global variable

def validate_paths(input_path, logo_path, output_folder):

    """
         Args:
            input_path (str): Path to the image or directory of images.
            logo_path (str): Path to the logo image to be detected.
            output_folder (str): Path to save the output image(s).

        Function:
            Validates the paths provided for input image(s), logo image, and output folder.

        Raises:
            FileNotFoundError: If input path, logo path, or output folder does not exist or is not a directory
            ValueError: If input file is not a valid image file, input folder does not contain any valid image file,
                        or logo file is not a valid image file.
    """

    if not (os.path.exists(input_path) and
            (os.path.isfile(input_path) or os.path.isdir(input_path))):
        raise ValueError("Invalid input path. Please provide a valid image file path or a folder path.")

    if not os.path.exists(output_folder) or not os.path.isdir(output_folder):
        raise FileNotFoundError("Output folder does not exist or is not a directory.")

    if not os.path.isfile(logo_path) or not logo_path.lower().endswith(VALID_IMAGE_EXTENSIONS):
        raise FileNotFoundError("Logo file does not exist or is not a valid image file.")

    if os.path.isfile(input_path) and not input_path.lower().endswith(VALID_IMAGE_EXTENSIONS):
        raise ValueError("Input file must be a valid image file.")
    elif os.path.isdir(input_path):
        input_files = [file.lower() for file in os.listdir(input_path)]
        if not any(file.endswith(VALID_IMAGE_EXTENSIONS) for file in input_files):
            raise ValueError("Input folder must contain at least one valid image file.")


This is our image loader which handles being given a single image or a folder of images.

In [72]:
def load_images(input_path, logo_path):

    """
        Args:
            input_path (str): Path to the input file or folder.
            logo_path (str): Path to the logo image.
        Returns:
            tuple: A tuple containing the loaded logo image and a list of paths to input images.
        Function:
            Loads the input image(s) and the logo image.
    """
    logo = cv2.imread(logo_path)

    if os.path.isfile(input_path):
        input_paths = [input_path]
    elif os.path.isdir(input_path):
        input_paths = [os.path.join(input_path, file) for file in os.listdir(input_path)
                       if file.lower().endswith(VALID_IMAGE_EXTENSIONS)]

    return logo, input_paths

Here we are trying to find keypoints and descriptors of our image using SIFT

In [73]:
def find_keypoints_and_descriptors(image, detector):

    """
        Args:
            image (numpy array): Input image.
            detector: Key point detector object.
        Returns:
            tuple: A tuple containing the keypoints and descriptors of the input image.
        Function:
            Finds keypoints and computes descriptors for the input image using the specified detector.
    """

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = detector.detectAndCompute(gray_image, None)
    return keypoints, descriptors


Here we match the descriptors between our logo and the image and only return the best matches.

In [74]:
def match_descriptors(des_logo, des_image):

    """
    Args:
        des_logo (numpy array): Descriptors of the logo image.
        des_image (numpy array): Descriptors of the input image.
    Returns:
        list: List of good matches between descriptors of the logo and input image.
    Function:
        Matches descriptors between the logo and input image, filtering out good matches based on distance ratio.
    """

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des_logo, des_image, k=2)
    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
    return good_matches

We resize our images then apply Gaussian Blur to remove noise and enhance feature extraction.

Then, we get the homography points using RANSAC and transform them to project them on the original image and make a mask around the logo. We then take the logo, replace it with white pixels, and use TELEA to reconstruct the missing part of the image.

In [75]:
def edit_image(image, logo, kp_logo, des_logo, detector):

    """
      Args:
          image (numpy.ndarray): Input image.
          logo (numpy.ndarray): Logo image.
          kp_logo: Keypoints of the logo image.
          des_logo: Descriptors of the logo image.
          detector: Key point detector object.

      Returns:
          numpy array: Processed image with logo removed.

      Function:
          Applies preprocessing, then edits the input image to remove the logo using keypoints, descriptors, and homography transformation,
          then uses TELEA method in inpaint to reconstruct the background of the removed area.
    """
    image = cv2.resize(image, (800,600))
    image = cv2.GaussianBlur(image, (5,5), 0)
    kp_image, des_image = find_keypoints_and_descriptors(image, detector)
    good_matches = match_descriptors(des_logo, des_image)

    if len(good_matches) > 10:
        src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_image[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)

        h, w = logo.shape[:2]
        logo_corners = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        transformed_logo_corners = cv2.perspectiveTransform(logo_corners, M)

        mask = np.zeros_like(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), dtype=np.uint8)
        cv2.fillPoly(mask, [np.int32(transformed_logo_corners)], 255)

        result_image = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)
    else:
        result_image = None

    return result_image

Our main function which manages all the above functions and puts them in place. It also saves the output of the SIFT if anything detected and outputs some facts about the performance of the algorithm.

In [76]:
def Process_image(input_path, logo_path, output_folder):

    """
        Args:
            input_path (str): Path to the input image(s) or directory containing input images.
            logo_path (str): Path to the logo image.
            output_folder (str): Path to save the processed output image(s).

        Function:
            This is our main function. It orchestrates the processing of input images to remove the logo,
            replace background and save the output.

        Steps:
            1. Validate Paths: Calls the validate_paths function to ensure the paths are valid.
            2. Load Images: Loads the logo image and input image(s) using the load_images function.
            3. Extract Features: Computes keypoints and descriptors for the logo image using the SIFT detector.
            4. Process Images: Iterates over each input image, removes the logo using the edit_image function, and saves the result.

        Output:
            The function generates processed output images with the logo removed. Each output image is saved in the specified output folder
            and follows the same name, format and dimensions as the input images.
    """

    validate_paths(input_path, logo_path, output_folder)
    logo, input_paths = load_images(input_path, logo_path)
    logo_gray = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    kp_logo, des_logo = sift.detectAndCompute(logo_gray, None)

    no_label_images = []
    total_images = 0
    no_label_count = 0

    for input_image_path in input_paths:
        total_images += 1
        image_name = os.path.basename(input_image_path)
        image = cv2.imread(input_image_path)
        result_image = edit_image(image, logo, kp_logo, des_logo, sift)
        if result_image is not None:
            output_image_path = os.path.join(output_folder, image_name)
            cv2.imwrite(output_image_path, result_image)
        else:
            no_label_count += 1
            no_label_images.append(image_name)

    percentage_label = ( (total_images - no_label_count) / total_images) * 100
    print(f"Percentage of images with label detected: {percentage_label:.2f}%")
    print(f"Number of images with no label detected: {no_label_count}")

    if no_label_count > 0:
        print("Images with no label detected:")
        print("\n".join(no_label_images))


Test Run

In [79]:
input_path = 'Insert your image or folder'
logo_path = 'insert the path to your logo image'
output_folder = 'Insert the directory you want to save your output to'

Process_image(input_path, logo_path, output_folder)

Percentage of images with label detected: 100.00%
Number of images with no label detected: 0


We made some error analysis as we got 93% accuracy on our first try. It appeared that images which had not been detected had higher light intensity than the other ones. Like image 27 and 42. Upon researching, we found that resizing and Gaussian Blur can help reduce the noise and enhance feature extraction.

In practice, we found that resizing alone got our score to jump to 100%. However, we included the Gaussian Blur step as we know that with our sample size, the score is not reliable and we better enhance preprocessing for images that are out of our test set.